# FLOW起動毎に必要な準備を行う

FLOWの立ち上げ毎に、ワークフロー実行のために必要なGINのユーザー情報を保持します。  
上から順番に実行してください。

## ◆◆◆開発メモ◆◆◆
ゆくゆくはSSOに対応し、コンテナ立ち上げ毎のユーザー名、パスワード、メールアドレスの入力を避けたい

## １．ユーザーの認証

この手順では、あなたのユーザ情報をシステムに認証させる手続きを行います。  
以下のセルを実行し、画面の表示に沿ってデータガバナンス機能に登録したユーザー名、パスワード、メールアドレスを入力してください。

In [ ]:
import json
from scripts import utils

# 以下の認証の手順で用いる、
# GINのドメイン名等をパラメタファイルから取得する
params = {}
with open(utils.fetch_param_file_path(), mode='r') as f:
    params = json.load(f)

In [ ]:
import os
import time
import getpass
import requests

from IPython.display import clear_output
from requests.auth import HTTPBasicAuth
from http import HTTPStatus

# 正常に認証が終わるまで繰り返し
while True:
    name = input("ユーザー名：")
    password = getpass.getpass("パスワード：")
    email = input("メールアドレス：")
    clear_output()
    
    # GIN API Basic Authentication
    # refs: https://docs.python-requests.org/en/master/user/authentication/
    
    # 既存のトークンがあるか確認する
    response = requests.get(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', auth=(name, password))
    tokens = response.json()

    # 既存のトークンがなければ作成する
    if len(tokens) < 1:
        response = requests.post(params['siblings']['ginHttp']+'api/v1/users/' + name + '/tokens', data={"name": "system-generated"} ,auth=(name, password))

    if response.status_code == HTTPStatus.OK or HTTPStatus.CREATED:
        tokens = response.json()
        clear_output()
        print("認証が正常に完了しました。次の手順へお進みください。")
        break
    else:
        clear_output()
        print("ユーザ名、またはパスワードが間違っています。\n恐れ入りますがもう一度ご入力ください。")

!git config --global user.name $name
!git config --global user.email $email
!cp ~/.gitconfig ~/WORKFLOW/PACKAGE/.gitconfig

## データ同期のための設定をする

この手順では、今の実行環境とデータガバナンス機能のリポジトリでデータの同期をとるための準備をします。  
以下を実行することで、システムがデータ同期の準備の手続きを行います。

In [ ]:
%%!
#!/bin/bash
if [ ! -e ~/.ssh/id_ed25519 ]; then
    # 鍵ペアが無ければ作成
    ssh-keygen -t ed25519 -N "" -f ~/.ssh/id_ed25519
fi

In [ ]:
# 公開鍵アップロード
# refs: https://github.com/gogs/docs-api/blob/master/Users/Public%20Keys.md#create-a-public-key
import os
import requests
import time
from http import HTTPStatus

import json
from scripts import utils

pubkey = !cat ~/.ssh/id_ed25519.pub

# 認証時に取得したトークンを使ってPOSTリクエスト
response = requests.post(
                params['siblings']['ginHttp']+'api/v1/user/keys?token=' + tokens[0]['sha1'],
                data={
                    "title": "system-generated-"+str(time.time()),
                    "key": pubkey[0]
                })
msg = response.json()

# コンテナを消す際にコンテナとつなぐための公開鍵も削除のため、
# パラメータとしてGINから発行された鍵IDを保存
if response.status_code == HTTPStatus.CREATED:
    # params.jsonへの追記（鍵ID）
    params['ginKeyId'] = str(response.json()['id'])
    with open(utils.fetch_param_file_path(), mode='w') as f:
        json.dump(params, f, indent=4)
    print('Public key is ready.')
elif msg['message'] == 'Key content has been used as non-deploy key':
    print('Public key is ready before time.')

In [ ]:
# sibling url をsshに変更する
import os
import tempfile
from datalad import api
from IPython.display import clear_output

remote_info = 'remote.txt'

with tempfile.TemporaryDirectory() as tmpdir:
    # 使い終わったらPython側で消してくれるTemporaryDirectoryを利用
    # refs: https://qiita.com/everylittle/items/aa7c6f612ff0a9db7f01
    tmppath = os.path.join(tmpdir, remote_info)
    !git remote show origin > $tmppath
    with open(tmppath, 'r') as f:
        lines = [s.strip() for s in f.readlines()]
        
        # FIXME: GitHub等リポジトリにも対応するためハードコーディングを解消
        ssh_push_url = lines[2].replace('Push  URL: http://dg02.dg.rcos.nii.ac.jp/', 'ssh://root@dg02.dg.rcos.nii.ac.jp:3001/') 

api.siblings(action='add', name='gin', url=ssh_push_url)
clear_output()
print('SSH connection is ready.')

## データ保存・変更の同期

ここまでの内容を保存・同期します。  
以下のセルを実行し、設定内容を同期してください。

In [ ]:
# SSHホスト（＝GIN）を信頼する設定
# ドメイン名がハードコーディングにつき要修正
with open('/home/jovyan/.ssh/config', mode='w') as f:
    f.write('host dg02.dg.rcos.nii.ac.jp\n\tStrictHostKeyChecking no\n\tUserKnownHostsFile=/dev/null\n')

In [ ]:
import papermill as pm

pm.execute_notebook(
    './base_datalad_save_push.ipynb',
    'push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '[GIN] 研究リポジトリ初期設定を完了', PATH='/home/jovyan/test.txt', IS_RECURSIVE=False)
)
print('データ同期が完了しました。お疲れ様でした！')
